# Ellipsoids equivariant scattering GNN experiments

## Generate data

In [ ]:
! python3.11 /scripts/python/generate_ellipsoid_dataset.py \
    --save_dir "<WORKING_DIR>/data/ellipsoids/" \
    --config /config/yaml_files/ellipsoids/experiment.yaml \
    --pq_h5_name pq_tensor_data_512.h5 \
    --random_seed 457892 \
    --num_samples 512 \
    --num_nodes_per_graph 128 \
    --knn_graph_k 5 \
    --abc_means 3.0 1.0 1.0 \
    --abc_stdevs 0.5 0.2 0.2 \
    --local_pca_kernel_fn gaussian \
    --laplacian_type sym_norm \
    --num_oversample_points 1024 \
    --k_laplacian 10 \
    --dirac_types max min \
    --modulation_scale 0.9 \
    --harmonic_bands 1-3,6-8,10-12 \
    --harmonic_band_weights 1.0 1.5 2.0 \
    --global_band_index 1 \
    --random_harmonic_k 16 \
    --random_harmonic_coeff_bounds 1.0 2.0 \
    --random_harmonic_smoothing_window 0 \
    --sing_vect_align_method column_dot

   python3 ../scripts/python/generate_ellipsoid_dataset.py \
    --save_dir "<WORKING_DIR>/data/ellipsoids/" \
    --config ../config/yaml_files/ellipsoids/experiment.yaml \
    --pq_h5_name pq_tensor_data_512.h5 \
    --random_seed 457892 \
    --num_samples 512 \
    --num_nodes_per_graph 128 \
    --knn_graph_k 5 \
    --abc_means 3.0 1.0 1.0 \
    --abc_stdevs 0.5 0.2 0.2 \
    --local_pca_kernel_fn gaussian \
    --laplacian_type sym_norm \
    --num_oversample_points 1024 \
    --k_laplacian 10 \
    --dirac_types max min \
    --modulation_scale 0.9 \
    --harmonic_bands 1-3,6-8,10-12 \
    --harmonic_band_weights 1.0 1.5 2.0 \
    --global_band_index 1 \
    --random_harmonic_k 16 \
    --random_harmonic_coeff_bounds 1.0 2.0 \
    --random_harmonic_smoothing_window 0 \
    --sing_vect_align_method column_dot



In [ ]:
# Open dataset and inspect a sample
import pickle
import sys
sys.path.append("")
from data_processing.ellipsoid_data_classes import EllipsoidDataset

pkl_path = "<WORKING_DIR>/data/ellipsoids/ellipsoid_dataset_512.pkl"

with open(pkl_path, "rb") as f:
    dataset = pickle.load(f)

print(dataset[0])
# print(dataset[0].y_node)

## Infogain Wavelet scales

### Scalar track

In [ ]:
! python3.11 /scripts/python/compute_infogain_scales.py \
    --config /config/yaml_files/ellipsoids/experiment.yaml \
    --h5_path <WORKING_DIR>/data/ellipsoids/pq_tensor_data_512.h5 \
    --output_dir <WORKING_DIR>/data/ellipsoids/infogain_scales/ \
    --track scalar \
    --feat_key x \
    --T 16 \
    --quantiles 0.25 0.5 0.75 \
    --divergence l1 \
    --device cuda

### Vector track

In [ ]:
! python3.11 /scripts/python/compute_infogain_scales.py \
    --config /config/yaml_files/ellipsoids/experiment.yaml \
    --h5_path <WORKING_DIR>/data/ellipsoids/pq_tensor_data_512.h5 \
    --output_dir <WORKING_DIR>/data/ellipsoids/infogain_scales/ \
    --track vector \
    --feat_key pos \
    --T 16 \
    --quantiles 0.25 0.5 0.75 \
    --divergence l1 \
    --device cuda

## Graph regression tasks

### Diameter estimation

#### 5-fold CV example

In [ ]:
! python3.11 ../scripts/python/main_training.py \
  --config_path ../config/yaml_files/ellipsoids/gcn.yaml \
  --dataset ellipsoids \
  --experiment_type kfold \
  --k_folds 5 \
  --n_epochs 16

In [ ]:
! python3.11 /scripts/python/summarize_experiments.py \
  --exp_dirs <WORKING_DIR>/experiments/ellipsoids/cv_test \
  --metrics mse \
  --agg mean \
  --decimals 4 \
  --out_tex <WORKING_DIR>/experiments/ellipsoids/cv_test/median_results_table.tex

#### Non-equivariant model

- Note that this model can learn an invariant prediction if not using --invariant_pred (i.e., using the 'agg' readout instead of 'mlp')...

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/escgnn.yaml \
    --use_dirac_nodes \
    --ablate_vector_track \
    --invariant_pred

#### Equivariant model

- With both scalar and vector features pointing to 'pos' attribute of Data objects, we need to include --scalar_feat_key (set to something besides 'pos') when using Diracs in order for them to be added as scalar features.

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/escgnn.yaml \
    --use_dirac_nodes \
    --scalar_feat_key x \
    --invariant_pred

#### EGNN

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/egnn.yaml

#### TFN

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/tfn_diameter.yaml \
    --invariant_pred

#### GIN

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/gin.yaml

#### GAT

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/gat.yaml

#### GCN

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/gcn.yaml

#### LEGS

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/legs.yaml \
    --use_dirac_nodes

## Node regression tasks

### Random-harmonic modulated normals

#### Non-equivariant model

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/escgnn.yaml \
    --task_key=vector_node_regression \
    --target_key=y_random_harmonic_normals \
    --target_dim=3 \
    --use_dirac_nodes \
    --ablate_vector_track

#### Equivariant model

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/escgnn.yaml \
    --task_key=vector_node_regression \
    --target_key=y_random_harmonic_normals \
    --target_dim=3 \
    --scalar_feat_key x \
    --use_dirac_nodes

#### EGNN

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/egnn.yaml \
    --task_key=vector_node_regression \
    --target_key=y_random_harmonic_normals \
    --target_dim=3

#### LEGS

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/legs.yaml \
    --task_key=vector_node_regression \
    --target_key=y_random_harmonic_normals \
    --target_dim=3 \
    --use_dirac_nodes

#### GIN

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/gin.yaml \
    --task_key=vector_node_regression \
    --target_key=y_random_harmonic_normals \
    --target_dim=3

#### GCN

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/gcn.yaml \
    --task_key=vector_node_regression \
    --target_key=y_random_harmonic_normals \
    --target_dim=3

#### GAT

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/gat.yaml \
    --task_key=vector_node_regression \
    --target_key=y_random_harmonic_normals \
    --target_dim=3

#### TFN

In [ ]:
! python3.11 ../scripts/python/main_training.py \
    --config=../config/yaml_files/ellipsoids/tfn_normals.yaml